In [ ]:
import sagemaker
from sagemaker import get_execution_role

In [ ]:
sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()
print(role)

In [ ]:
from sagemaker.sklearn.estimator import SKLearn


bucket = sagemaker_session.default_bucket()
s3_train_file = 'preprocessed/sentiments-preprocessed-wo-lemmatizing-with-stopwords-train-10percent-dataset.csv'
s3_test_file = 'preprocessed/sentiments-preprocessed-wo-lemmatizing-with-stopwords-test-10percent-dataset.csv'
script_path = 'startup_logisticRegression_model.py'

#Creates the SKLearn estimator
sklearn = SKLearn(
    framework_version='0.20.0',
    source_dir="src",
    entry_point=script_path,
    instance_type="ml.m5.xlarge",
    role=role,
    sagemaker_session=sagemaker_session
)

In [ ]:
#Start the training JOB in sagemaker
sklearn.fit({'train': 's3://{}/{}'.format(bucket, s3_train_file),
             'test': 's3://{}/{}'.format(bucket, s3_test_file)})

In [ ]:
#Deploy model to an endpoint
deployment = sklearn.deploy(initial_instance_count=1, instance_type="ml.t2.medium")

In [ ]:
import json

words = ["today is gonna be the day"]
payload = {"sentiments": words}

response = deployment.predict(
    json.dumps(payload), initial_args={"ContentType": "application/json", "Accept": "application/json"}
)

vecs = json.loads(response)
print(vecs)

In [ ]:
# Delete endpoint
deployment.delete_endpoint()